In [1]:
import pandas as pd
chunks = pd.read_csv("Datos_split/koi_light_curves_train.csv",header=None,iterator=True,chunksize=1011) #koi_light_curves

### Generar 5 archivos con la reducciones a 5, 10, 25, 55, 100, 250

In [2]:
for chunk in chunks:
    print(chunk.shape)

(1011, 71427)
(1011, 71427)
(1011, 71427)
(1011, 71427)


In [3]:
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
import csv

ds= [5,10,25,55,100]
for d in ds:
    model = IncrementalPCA(d)
    std = StandardScaler(with_mean=True,with_std=False) #sin sacar varianza
    
    ##################### TRAIN #####################
    chunks = pd.read_csv("Datos_split/koi_light_curves_train.csv",header=None,iterator=True,chunksize=1011)
    
    for chunk in chunks:
        X = np.nan_to_num(chunk.values)
        std.partial_fit(X)
    print("CenterScaler entrenado")
    
    chunks = pd.read_csv("Datos_split/koi_light_curves_train.csv",header=None,iterator=True,chunksize=1011)
    
    for chunk in chunks:
        X = np.nan_to_num(chunk.values)
        X_std = std.transform(X)
        model.partial_fit(X_std)
    print('PCA entrenado con', d, 'caracteristicas')

    #################### ARRIBA ENTRENAMIENTO ###########
    chunks = pd.read_csv("Datos_split/koi_light_curves_train.csv",header=None,iterator=True,chunksize=1011)
    doc_tr = open('koi_light_curves_PCA_train'+str(d)+'.csv', 'w') 
    tr = csv.writer(doc_tr)
    
    for chunk in chunks:
        X = np.nan_to_num(chunk.values)
        X_std = std.transform(X)
        tr.writerows(model.transform(X_std))
    doc_tr.close()
    print('Generacion de CSV koi_light_curves_PCA_train', d)
    
    ##################### VAL ########################
    chunks = pd.read_csv("Datos_split/koi_light_curves_val.csv",header=None,iterator=True,chunksize=1500)
    doc_val = open('koi_light_curves_PCA_val'+str(d)+'.csv', 'w') 
    val = csv.writer(doc_val)    
    
    for chunk in chunks:
        X_val = np.nan_to_num(chunk.values)
        X_val_std = std.transform(X_val)
        val.writerows(model.transform(X_val_std))
    doc_val.close()
    print('Generacion de CSV koi_light_curves_PCA_val', d)

    ##################### TEST ########################
    chunks = pd.read_csv("Datos_split/koi_light_curves_test.csv",header=None,iterator=True,chunksize=1500)
    doc_test = open('koi_light_curves_PCA_test'+str(d)+'.csv', 'w') 
    test = csv.writer(doc_test)    
    
    for chunk in chunks:
        X_test = np.nan_to_num(chunk.values)
        X_test_std = std.transform(X_test)
        test.writerows(model.transform(X_test_std))
    doc_test.close()
    print('Generacion de CSV koi_light_curves_PCA_test', d)
    
print('Proceso de reducción de dimensionalidad completo')

KeyboardInterrupt: 

In [11]:

d=10
chunks = pd.read_csv("Datos_split/koi_light_curves_train.csv",header=None,iterator=True,chunksize=1011) 
doc = open('koi_light_curves_PCA_'+str(d)+'.csv', 'w') 
a = csv.writer(doc)

model = IncrementalPCA(d)
for chunk in chunks:
    X = np.nan_to_num(chunk.values)
    model.partial_fit(X)
    a.writerows(model.transform(X))
doc.close()
print 'PCA aplicado con', d, 'caracteristicas'



for chunk in chunks:
    print(chunk.shape)    
    X = np.nan_to_num(chunk.values)
    a.writerows(model.transform(X))
b.close()


PCA aplicado con 10 caracteristicas


In [9]:
df = pd.read_csv('koi_light_curves_PCA_10.csv',header=None)

EmptyDataError: No columns to parse from file

## fourier

In [5]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
import csv
import pandas as pd

std = StandardScaler(with_mean=True,with_std=False) #sin sacar varianza
df_train = pd.read_csv("Datos_split/koi_light_curves_train.csv",header=None)

X_train = np.nan_to_num(df_train.values)
#fourier
Xtrain_fourier = np.abs(np.fft.fft(X_train))
std.fit(Xtrain_fourier)
Xtrain_std = std.transform(Xtrain_fourier)
print("CenterScaler entrenado")

df_val = pd.read_csv("Datos_split/koi_light_curves_val.csv",header=None)

X_val = np.nan_to_num(df_val.values)
#fourier
Xval_fourier = np.abs(np.fft.fft(X_val))
Xval_std = std.transform(Xval_fourier)
print("VAL escalado y fourier transformado")

df_test = pd.read_csv("Datos_split/koi_light_curves_test.csv",header=None)

X_test = np.nan_to_num(df_test.values)
#fourier
Xtest_fourier = np.abs(np.fft.fft(X_test))
Xtest_std = std.transform(Xtest_fourier)
print("TEST escalado y fourier transformado")

del df_train
del df_val
del df_test

CenterScaler entrenado
VAL escalado y fourier transformado
TEST escalado y fourier transformado


In [6]:
dims= [5,10,25,55,100]

for d in dims:
    model = IncrementalPCA(d)
    
    ##################### TRAIN #####################
    model.partial_fit(Xtrain_std)
    print('PCA entrenado con', d, 'caracteristicas')

    #################### ARRIBA ENTRENAMIENTO ###########
    doc_tr = open('koi_light_curves_FPCA_train'+str(d)+'.csv', 'w') 
    tr = csv.writer(doc_tr)

    tr.writerows(model.transform(Xtrain_std))
    doc_tr.close()
    print('Generacion de CSV koi_light_curves_PCA_train', d)
    
    ##################### VAL ########################
    doc_val = open('koi_light_curves_FPCA_val'+str(d)+'.csv', 'w') 
    val = csv.writer(doc_val)    
    
    val.writerows(model.transform(Xval_std))
    doc_val.close()
    print('Generacion de CSV koi_light_curves_PCA_val', d)

    ##################### TEST ########################
    doc_test = open('koi_light_curves_FPCA_test'+str(d)+'.csv', 'w') 
    test = csv.writer(doc_test)    

    test.writerows(model.transform(Xtest_std))
    doc_test.close()
    print('Generacion de CSV koi_light_curves_PCA_test', d)
    
print('Proceso de reducción de dimensionalidad completo')

PCA entrenado con 5 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 5
Generacion de CSV koi_light_curves_PCA_val 5
Generacion de CSV koi_light_curves_PCA_test 5
PCA entrenado con 10 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 10
Generacion de CSV koi_light_curves_PCA_val 10
Generacion de CSV koi_light_curves_PCA_test 10
PCA entrenado con 25 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 25
Generacion de CSV koi_light_curves_PCA_val 25
Generacion de CSV koi_light_curves_PCA_test 25
PCA entrenado con 55 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 55
Generacion de CSV koi_light_curves_PCA_val 55
Generacion de CSV koi_light_curves_PCA_test 55
PCA entrenado con 100 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 100
Generacion de CSV koi_light_curves_PCA_val 100
Generacion de CSV koi_light_curves_PCA_test 100
Proceso de reducción de dimensionalidad completo


## Resampled y luego fourier

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
import csv
import pandas as pd

std = StandardScaler(with_mean=True,with_std=False) #sin sacar varianza
df_train = pd.read_csv("Resample/koi_light_curves_resample_train.csv",header=None)

X_train = df_train.values
#fourier
Xtrain_fourier = np.abs(np.fft.fft(X_train))
std.fit(Xtrain_fourier)
Xtrain_std = std.transform(Xtrain_fourier)
print("CenterScaler entrenado")

df_val = pd.read_csv("Resample/koi_light_curves_resample_val.csv",header=None)

X_val = df_val.values
#fourier
Xval_fourier = np.abs(np.fft.fft(X_val))
Xval_std = std.transform(Xval_fourier)
print("VAL escalado y fourier transformado")

df_test = pd.read_csv("Resample/koi_light_curves_resample_test.csv",header=None)

X_test = df_test.values
#fourier
Xtest_fourier = np.abs(np.fft.fft(X_test))
Xtest_std = std.transform(Xtest_fourier)
print("TEST escalado y fourier transformado")

del df_train
del df_val
del df_test

CenterScaler entrenado
VAL escalado y fourier transformado
TEST escalado y fourier transformado


In [5]:
dims= [5,10,25,55,100,250]

for d in dims:
    model = IncrementalPCA(d)
    
    ##################### TRAIN #####################
    model.partial_fit(Xtrain_std)
    print('PCA entrenado con', d, 'caracteristicas')

    #################### ARRIBA ENTRENAMIENTO ###########
    doc_tr = open('koi_light_curves_FPCA_resampled_train'+str(d)+'.csv', 'w') 
    tr = csv.writer(doc_tr)

    tr.writerows(model.transform(Xtrain_std))
    doc_tr.close()
    print('Generacion de CSV koi_light_curves_PCA_train', d)
    
    ##################### VAL ########################
    doc_val = open('koi_light_curves_FPCA_resampled_val'+str(d)+'.csv', 'w') 
    val = csv.writer(doc_val)    
    
    val.writerows(model.transform(Xval_std))
    doc_val.close()
    print('Generacion de CSV koi_light_curves_PCA_val', d)

    ##################### TEST ########################
    doc_test = open('koi_light_curves_FPCA_resampled_test'+str(d)+'.csv', 'w') 
    test = csv.writer(doc_test)    

    test.writerows(model.transform(Xtest_std))
    doc_test.close()
    print('Generacion de CSV koi_light_curves_PCA_test', d)
    
print('Proceso de reducción de dimensionalidad completo')

PCA entrenado con 5 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 5
Generacion de CSV koi_light_curves_PCA_val 5
Generacion de CSV koi_light_curves_PCA_test 5
PCA entrenado con 10 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 10
Generacion de CSV koi_light_curves_PCA_val 10
Generacion de CSV koi_light_curves_PCA_test 10
PCA entrenado con 25 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 25
Generacion de CSV koi_light_curves_PCA_val 25
Generacion de CSV koi_light_curves_PCA_test 25
PCA entrenado con 55 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 55
Generacion de CSV koi_light_curves_PCA_val 55
Generacion de CSV koi_light_curves_PCA_test 55
PCA entrenado con 100 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 100
Generacion de CSV koi_light_curves_PCA_val 100
Generacion de CSV koi_light_curves_PCA_test 100
PCA entrenado con 250 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 250
Generacion de CSV koi_l

## si se completa con linea de tendencia

In [7]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
import csv
import pandas as pd

std = StandardScaler(with_mean=True,with_std=False) #sin sacar varianza
df_train = pd.read_csv("Linea_Tendencia/koi_light_curves_full_train.csv",header=None)

X_train = df_train.values
#fourier
Xtrain_fourier = np.abs(np.fft.fft(X_train))
std.fit(Xtrain_fourier)
Xtrain_std = std.transform(Xtrain_fourier)
print("CenterScaler entrenado")

df_val = pd.read_csv("Linea_Tendencia/koi_light_curves_full_val.csv",header=None)

X_val = df_val.values
#fourier
Xval_fourier = np.abs(np.fft.fft(X_val))
Xval_std = std.transform(Xval_fourier)
print("VAL escalado y fourier transformado")

df_test = pd.read_csv("Linea_Tendencia/koi_light_curves_full_test.csv",header=None)

X_test = df_test.values
#fourier
Xtest_fourier = np.abs(np.fft.fft(X_test))
Xtest_std = std.transform(Xtest_fourier)
print("TEST escalado y fourier transformado")

del df_train
del df_val
del df_test

CenterScaler entrenado
VAL escalado y fourier transformado
TEST escalado y fourier transformado


In [8]:
dims= [5,10,25,55,100]

for d in dims:
    model = IncrementalPCA(d)
    
    ##################### TRAIN #####################
    model.partial_fit(Xtrain_std)
    print('PCA entrenado con', d, 'caracteristicas')

    #################### ARRIBA ENTRENAMIENTO ###########
    doc_tr = open('koi_light_curves_FPCA_full_train'+str(d)+'.csv', 'w') 
    tr = csv.writer(doc_tr)

    tr.writerows(model.transform(Xtrain_std))
    doc_tr.close()
    print('Generacion de CSV koi_light_curves_PCA_train', d)
    
    ##################### VAL ########################
    doc_val = open('koi_light_curves_FPCA_full_val'+str(d)+'.csv', 'w') 
    val = csv.writer(doc_val)    
    
    val.writerows(model.transform(Xval_std))
    doc_val.close()
    print('Generacion de CSV koi_light_curves_PCA_val', d)

    ##################### TEST ########################
    doc_test = open('koi_light_curves_FPCA_full_test'+str(d)+'.csv', 'w') 
    test = csv.writer(doc_test)    

    test.writerows(model.transform(Xtest_std))
    doc_test.close()
    print('Generacion de CSV koi_light_curves_PCA_test', d)
    
print('Proceso de reducción de dimensionalidad completo')

PCA entrenado con 5 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 5
Generacion de CSV koi_light_curves_PCA_val 5
Generacion de CSV koi_light_curves_PCA_test 5
PCA entrenado con 10 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 10
Generacion de CSV koi_light_curves_PCA_val 10
Generacion de CSV koi_light_curves_PCA_test 10
PCA entrenado con 25 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 25
Generacion de CSV koi_light_curves_PCA_val 25
Generacion de CSV koi_light_curves_PCA_test 25
PCA entrenado con 55 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 55
Generacion de CSV koi_light_curves_PCA_val 55
Generacion de CSV koi_light_curves_PCA_test 55
PCA entrenado con 100 caracteristicas
Generacion de CSV koi_light_curves_PCA_train 100
Generacion de CSV koi_light_curves_PCA_val 100
Generacion de CSV koi_light_curves_PCA_test 100
Proceso de reducción de dimensionalidad completo


## LDA

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
y = (pd.read_csv("Datos_split/koi_labels_train.csv")["NExScI Disposition"].values == "CONFIRMED")*1
model = LDA()
model.fit(Xtrain_std,y)

/etc/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [10]:
#################### TRAIN ###########
doc_tr = open('koi_light_curves_FLDA_resampled_train'+str(d)+'.csv', 'w') 
tr = csv.writer(doc_tr)

tr.writerows(model.transform(Xtrain_std))
doc_tr.close()

##################### VAL ########################
doc_val = open('koi_light_curves_FLDA_resampled_val'+str(d)+'.csv', 'w') 
val = csv.writer(doc_val)    

val.writerows(model.transform(Xval_std))
doc_val.close()

##################### TEST ########################
doc_test = open('koi_light_curves_FLDA_resampled_test'+str(d)+'.csv', 'w') 
test = csv.writer(doc_test)    

test.writerows(model.transform(Xtest_std))
doc_test.close()

print('Proceso de reducción de dimensionalidad completo')

Proceso de reducción de dimensionalidad completo
